In [9]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

# Helper Clases / Functions

In [10]:
def checkAccuracy(model, dataloader):
    y_pred_tot = []
    y_sample_tot = []
    for i_batch, sample_batched in enumerate(dataloader):
        X_sample, y_sample = sample_batched
        # Flip axis first
        X_sample = torch.Tensor(np.moveaxis(X_sample.numpy(), -1, 0))
        X_sample, y_sample = Variable(X_sample), Variable(y_sample)
        scores = model(X_sample)
        #loss = loss_fn(scores, y_sample.long())
        #print('val loss: %f' % loss)
        _, y_pred = torch.max(scores, 1)
        y_pred_tot.append(y_pred.data.numpy())
        y_sample_tot.append(y_sample.data.numpy())
    acc = np.mean(np.concatenate(y_pred_tot) == np.concatenate(y_sample_tot))
    return acc
class Flatten(nn.Module):
    def forward(self, x):
        out, _ = x
        T, N, H = out.size() 
        return out.view(H, -1)

# Load Data

In [20]:
X_all = []
y_all = []
for i in range(8):
    file_path = './../project_datasets/A0' + str(i+1) + 'T_slice.mat'
    data = h5py.File(file_path, 'r')
    X = np.copy(data['image'])
    y = np.copy(data['type'])
    X = X[:, 0:23, :]
    X_all.append(X)
    y = y[0,0:X.shape[0]:1]
    y_all.append(y)
A, N, E, T = np.shape(X_all)
X_all = np.reshape(X_all, (A*N, E, T))
y_all = np.reshape(y_all, (-1))
y_all = y_all - 769
## Remove NAN
index_Nan = []
for i in range(A*N):
    for j in range(E):
        if (any(np.isnan(X_all[i,j])) == True):
            index_Nan.append(i)
X_all = np.delete(X_all, index_Nan, axis=0)
y_all = np.delete(y_all, index_Nan)
print (np.shape(X_all))
print (np.shape(y_all))

(2280, 23, 1000)
(2280,)


# Make DataLoaders

In [21]:
bs = 50
data = data_utils.TensorDataset(torch.Tensor(X_all), torch.Tensor(y_all))
dset = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [A*N-100-24, 50, 50])
dataloaders = {x: data_utils.DataLoader(dset[x], batch_size=bs, shuffle=True, num_workers=1) for x in ['train', 'val', 'test']}

# Define Model

In [22]:
class myLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myLSTM, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layer)
                            # batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        # h0 = Variable(torch.zeros(self.n_layer, x.size(1),
                                #   self.hidden_dim)).cuda()
        # c0 = Variable(torch.zeros(self.n_layer, x.size(1),
                                #   self.hidden_dim)).cuda()
        out, _ = self.lstm(x)
        out = out[-1, :, :]
        out = self.classifier(out)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            # Flip axis first
            X_sample = torch.Tensor(np.moveaxis(X_sample.numpy(), -1, 0))
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample)
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.numpy() == y_sample.data.numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

# Test the Model

In [23]:
dtype = torch.FloatTensor
hidden_dim = 100
num_classes = 4
num_epoches = 10
model = myLSTM(E, hidden_dim, 1, num_classes)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(num_epoches):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        # Flip axis first
        X_train = torch.Tensor(np.moveaxis(X_train.numpy(), -1, 0))
        X_train, y_train = Variable(X_train), Variable(y_train)
        # forward + backward + optimize
        out = model(X_train)
        print (out)
        print (out.size())
        print (y_train.size())
        loss = loss_fn(out, y_train.long())
        print('(%d batch) loss: %f' % (i, loss))
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_acc = model.check_accuracy(dataloaders['train'])
    val_acc = model.check_accuracy(dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epoches, train_acc, val_acc))
        

Variable containing:
 0.2203 -0.2453 -0.3081  0.0172
 0.1495 -0.1227 -0.1738 -0.0453
-0.2960 -0.0887 -0.3827 -0.1091
 0.5739 -0.2150 -0.1779  0.0570
-0.1596  0.3117 -0.1160  0.4199
-0.4519  0.3600 -0.2664  0.2657
 0.0435 -0.0735 -0.2059  0.0751
-0.0412  0.5323 -0.2088  0.0293
 0.2927  0.0531 -0.1833 -0.1989
 0.0477  0.0072 -0.0519 -0.1737
-0.3693 -0.0575  0.0854  0.3287
 0.1823 -0.2499 -0.1397  0.0258
-0.0026 -0.0261 -0.1165 -0.1375
 0.2373  0.0343 -0.0890  0.1260
 0.3349 -0.1221 -0.2317 -0.1338
-0.1510 -0.1396 -0.1578 -0.0455
-0.1560  0.4234 -0.1441  0.3211
-0.4211  0.2914 -0.2639  0.2054
 0.0987  0.1358 -0.1212 -0.2163
 0.0966 -0.1600 -0.2815  0.0377
-0.4738 -0.0890 -0.2148  0.0639
 0.1110 -0.0973 -0.3313 -0.1048
-0.0772 -0.2529  0.0031 -0.2731
-0.2825  0.5513 -0.2748  0.0987
-0.1194 -0.1219  0.0548 -0.0664
-0.1647  0.6295  0.0629  0.0914
 0.2961 -0.1342 -0.1816  0.0351
-0.2227 -0.1998  0.0643 -0.1476
-0.0561  0.2201  0.0839  0.2283
-0.2433  0.4491 -0.2152  0.3947
-0.3552  0.1097 -0.

Process Process-9:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/andrew/Desktop/Project/2ndtest/.env/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderI

KeyboardInterrupt: 